# Run py12box model
Run the AGAGE 12-box model

In [33]:
import os
import matplotlib.pyplot as plt
import setup
import core
from ipywidgets import widgets
from pandas import read_csv
from pathlib import Path

In [37]:
dir_path = Path('.').parent

In [38]:
#os.path.realpath(".")
input_dir = os.path.join(dir_path, "inputs")
case_dir = os.path.join(dir_path, "example")
species_info = read_csv(os.path.join(input_dir, "species_info.csv"),
                       index_col = "Species")
species_list = species_info.index.values

In [31]:
model_inputs = {"species": None,
                "mol_mass": None}

def set_species(species):
    mol_mass = species_info["Molecular mass (g/mol)"][species]
    model_inputs["mol_mass"] = mol_mass
    model_inputs["species"] = species
    print(model_inputs)

widgets.interact(set_species, species = species_list)

interactive(children=(Dropdown(description='species', options=('CFC-11', 'CFC-12', 'CFC-13', 'CFC-113', 'CFC-1…

<function __main__.set_species(species)>

In [32]:
species = "CFC-11"
mol_mass = 137.3688
time, emissions, ic, lifetime = setup.get_species_parameters(project_dir,
                                                             case_dir,
                                                             species)

TypeError: get_species_parameters() missing 1 required positional argument: 'species'

In [ ]:
i_t, i_v1, t, v1, OH, Cl, temperature = setup.get_model_parameters(input_dir,
                                                                   int(len(time) / 12))
F = setup.transport_matrix(i_t, i_v1, t, v1)
c_month, burden, emissions_out, losses, lifetimes = \
    core.model(ic=ic, q=emissions,
               mol_mass=mol_mass,
               lifetime=lifetime,
               F=F,
               temp=temperature,
               Cl=Cl, OH=OH)

In [ ]:
plt.plot(time, c_month[:, 0])
plt.plot(time, c_month[:, 3])
plt.ylabel("%s (pmol mol$^{-1}$)" % species)
plt.show()